## Lesson 4. Рекоммендательные системы на основе контента. Поиск похожих товаров и пользователей

1. Изучите структуру модуля src
2. Перенесите функции prefilter_items и postfilter_items из вебинара в модуль src.utils.py
3. Реализуйте функции get_similar_items_recommendation, get_similar_users_recommendation (они разбирались на вебинаре) и переместите в src.utils.py
4. Создайте модуль src.recommenders.py. Напищите код для класса ниже и положите его в src.recommenders.py
5. Проверьте, что все модули корректно импортируются
6. Если вы еще не прочитали статью о рекомендательных системах и поиске в hh.ru, то обязательно прочитайте

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items, get_similar_items_recommendation, get_similar_users_recommendation

In [3]:
data = pd.read_csv('../raw_data/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
item_features = pd.read_csv('../raw_data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


### Провекра функции prefilter_items

In [5]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train,item_features,['GROCERY']) #'GROCERY - Это для паримера. 

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 90386 to 5001


In [6]:
# Проверяем корректно ли отработала функция

# пропали товары, которые не покупали более 12 месяцев.
print(f"week_no max={data_train['week_no'].max()}, min ={data_train['week_no'].min()} ")
# пропали очень дорогие и очень дешевые товары
print(f"week_no max={data_train['sales_value'].max()}, min ={data_train['sales_value'].min()} ")

week_no max=98, min =86 
week_no max=30.0, min =1.0 


In [7]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,83647,197321,397896,480014,819210,819255,819308,819423,819549,819578,...,17381995,17827215,17900896,17900967,17901218,17901795,17902844,17957772,18000012,18105264
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [9]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [10]:
%%time

model = AlternatingLeastSquares(factors=20, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

100%|██████████| 15/15 [00:04<00:00,  3.64it/s, loss=0.0233]

Wall time: 4.14 s


## similar_users_recommendation

In [11]:
userid = 2500
get_similar_users_recommendation(userid, userid_to_id, id_to_userid,
                                 user_item_matrix, model, N=5)

[(3173, 23522.96290734811),
 (4620, 16505.994083354617),
 (2545, 10871.822995413904),
 (1575, 1423.6296659161555),
 (2521, 17892.787924530603)]

## similar_items_recommendation

In [12]:
userid = 2500
get_similar_items_recommendation(userid, data_train,
                                 itemid_to_id=itemid_to_id,
                                 id_to_itemid=id_to_itemid,
                                 model=model, N=5)

[950894, 1007512, 887325, 13918661, 1027168]

### Задание 4 - 6.

Проверка, что все работает

In [13]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [14]:
recommender = MainRecommender(data_train)

100%|██████████| 5001/5001 [00:00<00:00, 139293.00it/s]


In [15]:
userid = 2500
recommender.get_similar_items_recommendation(userid, N=5)

[916122, 1051468, 838261, 871076, 844179]